**Problem:**

You are given the following dataset:
1. **Audible Data** : https://1drv.ms/u/s!AiqdXCxPTydhoog8ckLN-6Cw55fzIg?e=EWgZ5d

Your task is to:
- Find the problems with the datasets.
- Define the Data Quality Dimensions.
- Try to clean the datasets.

## Problem with the dataset:
- `name`:
    - Some books has the version in different formats.
        - [x] with hast-tag like "Geronimo Stilton #11 & #12".
        - [x] "Magic Tree House Collection: Books 9-16".
        - [x] "The 39 Clues, Book 6"
    - â€™, Ã¤, Ã¼ values
        - [x] Some rows (9, 34, 157, 162, 170...) have this strange part.


- `author` & `narrator`:
    - [x] Every value starts with "Writtenby:" e.g., "Writtenby:GeronimoStilton"
    - [x] Some values represent 2 or more authors, e.g., "Writtenby:JuliaDonaldson,AxelScheffler"
    - [x] Some values also have the strange part, e.g., "Writtenby:FranciscoDÃ­azValladares"
    - [x] First name and the last are not separated with a white space, e.g. "Writtenby:NicolasGorny".
    - [x] Some additional informations are also inclused in some values. E.g., "Writtenby:AndrewPeterson-editor,JonathanRogers,N.D.Wilson,"
    - [x] There are no proper names in narrator, e.g., "Narratedby:uncredited".


- `time`:
    - [x] The values are combination of total hour and minutes, e.g., "2 hrs and 20 mins", "10 hrs", "22 mins"


- `releasedate`:
    - [x] There are 2 types entries, "08-04-2008" and "13-01-10"
    - [x] The dtype is object, if we convert to datetime object, then the above will be resolved.


- `language`
    - [x] Some values are in title formed and some are in lower case, e.g. "English" and "german".


- `stars`
    - [x] Total ratings informations are included along with average stars, e.g. "5 out of 5 stars34 ratings"
    - [x] The highest rating is 5 and lower is 1. How many avg. stargs got a book is included as long form.
    - [x] Some empty values are represented as "Not rated yet". For that, we can assume total ratings = 0 and avg. ratings = 0.


- `price`
    - [x] There is a value, "Free". This also change the dtype of the column.

In [ ]:
import numpy as np
import pandas as pd

import re

In [7]:
# you can read the dataset by upload the excel file; here I am using same file's link
df = pd.read_csv("/content/audible_uncleaned.csv")
df.head()
df.shape

(87489, 8)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [8]:
df.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [9]:
# convert the "releasedate" col to datetime object
df["releasedate"] = pd.to_datetime(df["releasedate"]).copy()
# remove the value "free" from the col price. This will automatically
# convert to the float values.
df["price"] = df["price"].replace("Free", 0).copy()

/tmp/ipython-input-2371264084.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["releasedate"] = pd.to_datetime(df["releasedate"]).copy()


In [10]:
# now check again the dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         87489 non-null  object        
 1   author       87489 non-null  object        
 2   narrator     87489 non-null  object        
 3   time         87489 non-null  object        
 4   releasedate  87489 non-null  datetime64[ns]
 5   language     87489 non-null  object        
 6   stars        87489 non-null  object        
 7   price        87489 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 5.3+ MB


In [11]:
def find_total_ratings(text):
    # if NaN, then return as it is
    if pd.isna(text):
        return text

    # create the pattern to find the total ratings
    tol_rattings_pattern = re.compile(r"[\d]+\sratings")
    # get the result
    result = re.findall(tol_rattings_pattern, text)

    try:
        # if we find the total ratings, then convert to int and return it
        if len(result) > 0:
            return int(result[0].split(" ")[0])
        # if it is the "Not rated yet", return 0
        elif "Not rated yet":
            return 0
        # else return the value as it is
        else:
            return text
    # if we got any error, also return the value as it is
    except Exception as e:
        print(e)
        return text

In [12]:
# find out the total counted ratings
df["total_ratings"] = df["stars"].apply(lambda x: find_total_ratings(x))
df["avg_stars"]     = df["stars"].apply(lambda x: float(x.split(" ")[0]) if x != "Not rated yet" else 0)
# now drop the "stars" column
df = df.drop(columns = ["stars"]).copy()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           87489 non-null  object        
 1   author         87489 non-null  object        
 2   narrator       87489 non-null  object        
 3   time           87489 non-null  object        
 4   releasedate    87489 non-null  datetime64[ns]
 5   language       87489 non-null  object        
 6   price          87489 non-null  float64       
 7   total_ratings  87489 non-null  int64         
 8   avg_stars      87489 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 6.0+ MB


In [13]:
# convert the "language" col to titlecase
df["language"] = df["language"].str.title().copy()

In [14]:
def get_total_time(text):
    hrs = 0
    mins = 0
    # extract the total hours
    if "hrs" in text:
        hrs = int(text.split("hrs")[0].strip())
    # extract the total minutes
    if "mins" in text:
        is_mins = text.split("mins")[0].split(" ")
        # if the text is combination of hours and minutes, then will get the more elements
        if len(is_mins) > 2:
            mins = is_mins[-2]
        # if the text contains only minutes, e.g. "22 mins", then will get 2 length list
        else:
            mins = is_mins[0]
        mins = int(mins.strip())

    # calculate total minutes and return that
    return hrs * 60 + mins

In [15]:
# get the total time in minutes
df["time"] = df["time"].apply(lambda x: get_total_time(x)).copy()

In [16]:
# next we will remove the strange characters from the "name", "author" and "narrator" cols.
# When we will do the operation, some values of "name" col will become to NaN. So first extract
# those records which will become to NaN. After removing the strange values, we will replace
# those newly created NaN values with the old ones
nan_created_values = df[df["name"].str.encode('ascii', 'ignore').isnull()]

In [17]:
# some "narrator" values are completly packed with strange values like this example.
# So, while removing this strange values, the calues will convert to empty strings
df["narrator"][87192]

'Narratedby:小野田英一'

In [18]:
# Now, remove the starnge parts from the "name", "author" and "narrator" columns
df["name"]     = df["name"].str.encode('ascii', 'ignore').str.decode('ascii').copy()
df["author"]   = df["author"].str.encode('ascii', 'ignore').str.decode('ascii').copy()
df["narrator"] = df["narrator"].str.encode('ascii', 'ignore').str.decode('ascii').copy()

In [19]:
df["narrator"][87192]

'Narratedby:'

In [ ]:
df.isnull().sum()

name             53
author            0
narrator          0
time              0
releasedate       0
language          0
price             0
total_ratings     0
avg_stars         0
dtype: int64

In [20]:
# now remove the NaN values of the "name" column with the old ones
for index in nan_created_values.index:
    df["name"][index] = nan_created_values["name"][index]

In [21]:
df.isnull().sum()

,0
name,0
author,0
narrator,0
time,0
releasedate,0
language,0
price,0
total_ratings,0
avg_stars,0


In [24]:
# again NaN values will be created when you will gp to remove the versions from the "name" col.
# So, again will be same strategy, first will store those records which will become to NaN.
# After the operation, we will replace those NaN values with the old ones
book_version_pattern = re.compile(r"(\s#?\d+\s?(&|and)?\s?#?(\d+)?)|(:?,?\sBooks?\s\d+\s?-?(\d+)?)")
nan_books_version = df[df["name"].str.replace(book_version_pattern, "",regex=True).isnull()]

In [25]:
# remove the versions from the "name"
book_version_pattern = re.compile(r"(\s#?\d+\s?(&|and)?\s?#?(\d+)?)|(:?,?\sBooks?\s\d+\s?-?(\d+)?)")
df["name"] = df["name"].str.replace(book_version_pattern, "",regex=True).copy()

In [26]:
df.isnull().sum()

,0
name,0
author,0
narrator,0
time,0
releasedate,0
language,0
price,0
total_ratings,0
avg_stars,0


In [28]:
for index in nan_books_version.index:
    df["name"][index] = nan_books_version["name"][index]

In [29]:
df.isnull().sum()

,0
name,0
author,0
narrator,0
time,0
releasedate,0
language,0
price,0
total_ratings,0
avg_stars,0


In [30]:
# remove prefix "Writtenby:" & "Narratedby:" from "author" & "narrator" columns
df["narrator"] = df["narrator"].str.replace("Narratedby:", "").copy()
df["author"]   = df["author"].str.replace("Writtenby:", "").copy()

In [32]:
# replace the "uncredited" from the "narrator" col with NaN value
df["narrator"] = df["narrator"].replace("uncredited", np.nan).copy()

In [ ]:
# split those values which have 2 or more author or narrator
df["author"] = df["author"].apply(lambda x: x.split(",") if "," in x else [x]).copy()
df["narrator"] = df["narrator"].apply(lambda x: x if pd.isna(x) else x.split(",") if "," in x else [x]).copy()

In [ ]:
df.isnull().sum()

name               0
author             0
narrator         326
time               0
releasedate        0
language           0
price              0
total_ratings      0
avg_stars          0
dtype: int64

In [33]:
# remove the more informations from the "author" col because most of the values
# has no such info. Also, while modifying this col before, some blank strings
# (e.g. record no. 77) created. That strings will be removed here. Also, here
# will separate the first name and the last names. Previously those are not
# separated with a white space.

def separate_first_last_name(text):
    # create the regex pattern
    pattern = re.compile(r"[A-Z]+[a-z]+")
    # join the first & last name with a white space and return it
    return " ".join(re.findall(pattern, text))

def remove_more_info(text_list: list):
    if type(text_list) != list and pd.isna(text_list):
        return text_list

    # create the re pattern to remove the more info
    pattern = re.compile(r"-\w+")
    result = []

    # loop through the each values
    for item in text_list:
        # remove the empty string
        if len(item) != 0:
            # remove the more info
            re_result = re.sub(pattern, "", item)
            # separate the first and the last name
            result.append(separate_first_last_name(re_result))

    # if the result list is empty, then return NaN or return as it is
    return np.nan if len(result) <= 0 else result

In [ ]:
df["author"]   = df["author"].apply(lambda x: remove_more_info(x)).copy()
df["narrator"] = df["narrator"].apply(lambda x: remove_more_info(x)).copy()

In [ ]:
df.head()

,name,author,narrator,time,releasedate,language,price,total_ratings,avg_stars
0,Geronimo Stilton,[Geronimo Stilton],[Bill Lobely],140,2008-04-08,English,468.0,34,5.0
1,The Burning Maze,[Rick Riordan],[Robbie Daymond],788,2018-01-05,English,820.0,41,4.5
2,The Deep End,[Jeff Kinney],[Dan Russell],123,2020-06-11,English,410.0,38,4.5
3,Daughter of the Deep,[Rick Riordan],[Soneela Nankani],676,2021-05-10,English,615.0,12,4.5
4,The Lightning Thief: Percy Jackson,[Rick Riordan],[Jesse Bernstein],600,2010-01-13,English,820.0,181,4.5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           87489 non-null  object        
 1   author         84180 non-null  object        
 2   narrator       83421 non-null  object        
 3   time           87489 non-null  int64         
 4   releasedate    87489 non-null  datetime64[ns]
 5   language       87489 non-null  object        
 6   price          87489 non-null  float64       
 7   total_ratings  87489 non-null  int64         
 8   avg_stars      87489 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 6.0+ MB


In [34]:
# ok, for now it is done. For further use or analysis, you have to save the cleaned
# dataframe. We will not use the csv format to save because it takes more disk
# space and also the csv file can't retain the dtypes of the features of your
# dataframe. So you can store this file in one of these file formats: pickle,
# feather and parquet. Here I am using the pickle format.
df.to_pickle("audiable_cleaned.pickle")

In [35]:
# below syntax is to read the pickle file in pandas.
pd.read_pickle("audiable_cleaned.pickle").head()

,name,author,narrator,time,releasedate,language,price,total_ratings,avg_stars
0,Geronimo Stilton,GeronimoStilton,BillLobely,140,2008-04-08,English,468.00,34,5.0
1,The Burning Maze,RickRiordan,RobbieDaymond,788,2018-01-05,English,820.00,41,4.5
2,The Deep End,JeffKinney,DanRussell,123,2020-06-11,English,410.00,38,4.5
3,Daughter of the Deep,RickRiordan,SoneelaNankani,676,2021-05-10,English,615.00,12,4.5
4,The Lightning Thief: Percy Jackson,RickRiordan,JesseBernstein,600,2010-01-13,English,820.00,181,4.5
